## Computing Stats

In [1]:
# from model.losses import Stats 
import torch
import torchvision 
import os 
from PIL import Image
import torch
from torch import nn
from torch.nn import functional as F
from functorch import vmap 
from torchmetrics import StructuralSimilarityIndexMeasure
import lpips

In [ ]:
# Job ID          Username Queue    Jobname    SessID NDS TSK Memory Time  S Time   iter 530pm
# --------------- -------- -------- ---------- ------ --- --- ------ ----- - -----  ----
# 4459543.pbsha.* svsamban gpu      curls-hse3 233009   1   8   16gb 15:00 R 07:06  337 
# 4459552.pbsha.* svsamban gpu      curls-bse3 101567   1   8   24gb 35:00 R 06:51  182
# 4459556.pbsha.* svsamban gpu      toby-bse3  244690   1   8   16gb 35:00 R 06:35  163
# 4459557.pbsha.* svsamban gpu      toby-hse3  241595   1   8   16gb 35:00 R 06:36  340
# 4460675.pbsha.* svsamban gpu      toby-bt    174799   1   8   16gb 48:00 R 02:28  261
# 4461096.pbsha.* svsamban gpu      hnerf-c     29180   1   8   24gb 33:00 R 01:39  
# 4461097.pbsha.* svsamban gpu      nerfies-c   29399   1   8   24gb 24:00 R 01:39
# 4461637.pbsha.* svsamban gpu      hnerf-t    219157   1   8   24gb 33:00 R 00:42
# 4461638.pbsha.* svsamban gpu      nerfies-t  142863   1   8   24gb 24:00 R 00:40

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:

class Stats(nn.Module):
    def __init__(self, lpips=False):
        self.lpips = lpips
        super().__init__()

    def error_mse(self, im_pred, im_gt, mask = None):
        """
        Computes MSE metric. Optionally applies mask.
        """
        # Linearize.
        im_pred = im_pred[..., :3].reshape(-1, 3)
        im_gt = im_gt[..., :3].reshape(-1, 3)

        # Mask?
        if mask is not None:
            # mask = mask.flatten()
            # im_pred = im_pred[mask, :]
            # im_gt = im_gt[mask, :]

            # Use multiplication method as described in paper
            mask = mask.reshape(-1,1)
            im_pred = im_pred * mask[..., None]
            im_gt = im_gt * mask[..., None]

        mse = (im_pred - im_gt) ** 2
        return mse.mean()

    def error_psnr(self, im_pred, im_gt, mask=None):
        """
        Computes PSNR metric. Optionally applies mask.
        Assumes floats [0,1].
        """
        mse = self.error_mse(im_pred, im_gt, mask)
        # https://en.wikipedia.org/wiki/Peak_signal-to-noise_ratio
        return 20 * torch.log10(torch.tensor(1.0)) - 10 * torch.log10(mse), mse


    def error_ssim(self, im_pred, im_gt, pad=0, pad_mode='linear_ramp'):
        """Compute the LPIPS metric."""
        ssim = StructuralSimilarityIndexMeasure(data_range=1.0)
        return ssim(im_pred, im_gt)

    def error_lpips(self, im_pred, im_gt, mask=None, metric=None):
        """
        Computes LPIPS metric. Optionally applies mask.
        """
        # Mask?
        if mask is not None:
            print(mask.shape)
            mask = mask.reshape(im_pred.shape[0], im_pred.shape[1], 1).repeat(3, axis=2)
            print(mask.shape)
            im_pred = im_pred * mask
            im_gt = im_gt * mask

        # To torch.
        device = 'cuda'
        print(im_gt.shape, im_pred.shape)
        im_pred = (im_pred).to(device)
        im_gt = (im_gt).to(device)
        print(im_gt.shape, im_pred.shape)
        # Make metric.
        if metric is None:
            # best forward scores
            metric_a = lpips.LPIPS(net='alex').to(device)
            # # closer to "traditional" perceptual loss, when used for optimization
            metric_v = lpips.LPIPS(net='vgg').to(device)

        # Compute metric.
        loss_a = metric_a(im_pred, im_gt)
        loss_v = metric_v(im_pred, im_gt)

        return loss_a.item(), loss_v.item()

    def forward(self, im_pred, im_gt, mask=None):
        psnr, mse = self.error_psnr(im_pred, im_gt, mask=mask)
        ssim = self.error_ssim(im_pred, im_gt)
        if self.lpips:
            lpips_alex, lpips_vgg = self.error_lpips(im_pred, im_gt, mask=mask)
            stats = {
                'psnr': psnr,
                'lpips_alex': lpips_alex,
                'lpips_vgg': lpips_vgg,
                'ssim': ssim
            }
        else: 
            stats = {
                'psnr': psnr,
                'ssim': ssim
            }
        return stats

##curls

In [6]:

GT = torchvision.transforms.functional.to_tensor(Image.open('./data/DIY/curls/scan/image/000038.png'))
folder = 'ebse3_trainedforever'
num = '236'
pred = torchvision.transforms.functional.to_tensor(Image.open('./out/DIY/cluster/DIY/curls_' + folder + '/rendering/'+num+'000_vis/0038_unisurf.png'))

get_stats = Stats(lpips=False)

print(get_stats(pred.unsqueeze(0), GT.unsqueeze(0)))


{'psnr': tensor(34.8114), 'ssim': tensor(0.7879)}


##toby

In [5]:

GT = torchvision.transforms.functional.to_tensor(Image.open('/ubc/cs/research/kmyi/svsamban/research/unisurf/data/DIY/toby/scan/image/000040.png'))
folder = 'ebt'
num = '316'
pred_file = '/ubc/cs/research/kmyi/svsamban/research/unisurf/out/DIY/toby_' + folder + '/rendering/'+num+'000_vis/0040_unisurf.png'
pred = torchvision.transforms.functional.to_tensor(Image.open(pred_file))

get_stats = Stats(lpips=False)

print(get_stats(pred.unsqueeze(0), GT.unsqueeze(0)))


{'psnr': tensor(24.5979), 'ssim': tensor(0.4868)}


In [21]:
#METANLR 

folder = 'se3'

pic ='/ubc/cs/research/kmyi/svsamban/research/metanlrpp/logs/MYcurls_'+folder+'/meshes/model_final/test_benchmark/f000000_v038_color_pred.png'
pred = torchvision.transforms.functional.to_tensor(Image.open(pic))

GT = torchvision.transforms.functional.to_tensor(Image.open('./data/DIY/curls/scan/image/000038.png'))
GT = torchvision.transforms.Resize(pred.shape[1:])(GT)
mask = torchvision.transforms.functional.to_tensor(Image.open('./data/DIY/curls/scan/mask/000038.png'))
mask = torchvision.transforms.Resize(pred.shape[1:])(mask)#.squeeze()
get_stats = Stats(lpips=False)
print(get_stats(pred.unsqueeze(0), GT.unsqueeze(0), mask=mask))


{'psnr': tensor(11.2125), 'ssim': tensor(0.3147)}


In [25]:
#METANLR 

folder = 'control'

pic ='/ubc/cs/research/kmyi/svsamban/research/metanlrpp/logs/MYtoby_'+folder+'/meshes/model_final/test_benchmark/f000000_v040_color_pred.png'
pred = torchvision.transforms.functional.to_tensor(Image.open(pic))

GT = torchvision.transforms.functional.to_tensor(Image.open('./data/DIY/toby/scan/image/000040.png'))
GT = torchvision.transforms.Resize(pred.shape[1:])(GT)
mask = torchvision.transforms.functional.to_tensor(Image.open('./data/DIY/toby/scan/mask/000040.png'))
mask = torchvision.transforms.Resize(pred.shape[1:])(mask)#.squeeze()
get_stats = Stats(lpips=False)
print(get_stats(pred.unsqueeze(0), GT.unsqueeze(0), mask=mask))


{'psnr': tensor(19.9665), 'ssim': tensor(0.3065)}
